In [9]:
import ee
import geemap
import pandas as pd
import gpxpy
import geopandas as gpd
from shapely.geometry import Point, LineString

# Initialize Earth Engine
ee.Initialize()

# Load the terrain dataset for Arizona
terrain_dataset = ee.Image("USGS/SRTMGL1_003")

# Define a region of interest (ROI) for Arizona
arizona_roi = ee.Geometry.Polygon([
    [-114.8183, 32.5084],
    [-109.0452, 32.5084],
    [-109.0452, 37.0043],
    [-114.8183, 37.0043]
])

# Read the GPX file
gpx_file = "GPX_data/pass_01.gpx"
gpx_data = gpxpy.parse(open(gpx_file))

# Extract the data from the GPX file and store it in a Pandas DataFrame
gpx_df = pd.DataFrame([
    {
        "latitude": p.latitude,
        "longitude": p.longitude,
        "elevation": p.elevation,
        "name": p.name if p.name else "Unnamed"
    }
    for track in gpx_data.tracks
    for segment in track.segments
    for p in segment.points
])

# Center the map on Arizona
Map = geemap.Map(center=[34.5, -112], zoom=7)

# Add the terrain layer with a colorful palette
terrain_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']  # Custom color palette
}
Map.addLayer(terrain_dataset, terrain_vis, "Terrain")

# Convert the DataFrame to a GeoPandas DataFrame
geometry = [Point(lon, lat) for lon, lat in zip(gpx_df['longitude'], gpx_df['latitude'])]
gpx_gdf = gpd.GeoDataFrame(gpx_df, geometry=geometry, crs="EPSG:4326")

# Create a LineString geometry from the points
line_string = LineString(list(gpx_gdf['geometry']))

# Convert the LineString geometry to Earth Engine format
coords_list = list(line_string.coords)
ee_coords_list = [[coord[0], coord[1]] for coord in coords_list]
polyline_feature = ee.Geometry.LineString(ee_coords_list)

# Add the GPX points as a polyline with labels
polyline_vis = {
    'color': 'red',  # Change the color to any valid color name or RGB value
    'width': 3
}

Map.addLayer(polyline_feature, polyline_vis, "GPX Path")

# Display the map
Map

Map(center=[34.5, -112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…